# Exceptions

## try and except

In [1]:
# Basic try/except: catch errors and continue
try:
    result = 10 / 2
    print(f"Result: {result}")
except ZeroDivisionError:
    print("Cannot divide by zero")

# This would raise ZeroDivisionError without try/except
try:
    result = 10 / 0
    print(f"Result: {result}")
except ZeroDivisionError:
    print("Caught: Cannot divide by zero")

# Catching by exception type lets you handle specific errors
values = [1, 0, "two", 4]
for v in values:
    try:
        print(f"10 / {v!r} = {10 / v}")
    except ZeroDivisionError:
        print(f"10 / {v!r} -> division by zero")
    except TypeError:
        print(f"10 / {v!r} -> unsupported type")

Result: 5.0
Caught: Cannot divide by zero
10 / 1 = 10.0
10 / 0 -> division by zero
10 / 'two' -> unsupported type
10 / 4 = 2.5


## except with multiple types and else/finally

In [2]:
# else runs only when no exception occurred
try:
    x = int("42")
except ValueError:
    print("Invalid number")
else:
    print(f"Parsed successfully: {x}")

# finally runs always (cleanup, release resources)
try:
    f = open("/tmp/demo_exception.txt", "w")
    f.write("hello")
except OSError as e:
    print(f"IO error: {e}")
finally:
    if 'f' in dir() and f and not f.closed:
        f.close()
    print("Cleanup done (finally always runs)")

# Catching multiple exception types in one except
def safe_divide(a, b):
    try:
        return a / b
    except (TypeError, ZeroDivisionError) as e:
        return f"Error: {type(e).__name__}"

print(f"safe_divide(10, 2): {safe_divide(10, 2)}")
print(f"safe_divide(10, 0): {safe_divide(10, 0)}")
print(f"safe_divide(10, 'x'): {safe_divide(10, 'x')}")

Parsed successfully: 42
Cleanup done (finally always runs)
safe_divide(10, 2): 5.0
safe_divide(10, 0): Error: ZeroDivisionError
safe_divide(10, 'x'): Error: TypeError


## raise: raising exceptions

In [3]:
# raise without arguments re-raises the current exception (in an except block)
# raise SomeError("message") raises a new exception

def require_positive(n):
    if n <= 0:
        raise ValueError(f"Expected positive number, got {n}")
    return n

print(f"require_positive(5): {require_positive(5)}")
try:
    require_positive(-1)
except ValueError as e:
    print(f"Caught: {e}")

# Raising a different exception (chaining can be done with "from e")
def parse_score(s):
    try:
        return int(s)
    except ValueError as e:
        raise TypeError(f"Score must be a number-like string, got {s!r}") from e

try:
    parse_score("abc")
except TypeError as e:
    print(f"Caught: {e}")
    print(f"Cause: {e.__cause__}")

require_positive(5): 5
Caught: Expected positive number, got -1
Caught: Score must be a number-like string, got 'abc'
Cause: invalid literal for int() with base 10: 'abc'


## Custom exception classes

In [4]:
# Define custom exceptions by subclassing Exception
class ValidationError(Exception):
    """Raised when input fails validation."""
    pass

class ConfigError(Exception):
    """Raised when configuration is invalid."""
    def __init__(self, message, key=None):
        super().__init__(message)
        self.key = key

# Use custom exceptions for clearer error handling
def set_threshold(value):
    if not 0 <= value <= 1:
        raise ValidationError(f"Threshold must be in [0, 1], got {value}")
    return value

try:
    set_threshold(1.5)
except ValidationError as e:
    print(f"Validation failed: {e}")

try:
    raise ConfigError("Missing required key", key="api_key")
except ConfigError as e:
    print(f"ConfigError: {e}, key={e.key}")

Validation failed: Threshold must be in [0, 1], got 1.5
ConfigError: Missing required key, key=api_key


## Common built-in exceptions (quick reference)

In [5]:
# Often used in data/ML code:
# - ValueError: invalid value (e.g. wrong shape, negative where positive expected)
# - TypeError: wrong type (e.g. string instead of number)
# - KeyError: missing dict key
# - IndexError: list index out of range
# - FileNotFoundError, OSError: file/IO issues

try:
    d = {"a": 1}
    x = d["b"]
except KeyError as e:
    print(f"KeyError: {e}")

try:
    lst = [1, 2, 3]
    x = lst[10]
except IndexError as e:
    print(f"IndexError: {e}")

# Prefer asking forgiveness over permission when appropriate
d = {"a": 1, "b": 2}
key = "b"
value = d.get(key)  # None if missing; no exception
print(f"d.get({key!r}): {value}")
value = d.get("z", 0)  # default 0 if missing
print(f"d.get('z', 0): {value}")

KeyError: 'b'
IndexError: list index out of range
d.get('b'): 2
d.get('z', 0): 0
